In [1]:
# imports

import os
from dotenv import load_dotenv
from openai import OpenAI
import anthropic
from IPython.display import Markdown, display, update_display
import gradio as gr # oh yeah!

In [2]:
# Connect to OpenAI, Anthropic and Google; comment out the Claude or Google lines if you're not using them

openai = OpenAI()

claude = anthropic.Anthropic()


In [3]:
system_message = "your an helpfull assistent"

In [4]:
# Let's wrap a call to GPT-4o-mini in a simple function

def message_gpt(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
      ]
    completion = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=messages,
    )
    return completion.choices[0].message.content

In [5]:
message_gpt("what is today's date ? ")

"Today's date is October 4, 2023."

In [6]:
def shout(text):
    return text.upper()

In [7]:
gr.Interface(fn= shout, inputs="textbox", outputs= "textbox").launch() 

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


In [8]:
force_dark_mode = """
function refresh() {
    const url = new URL(window.location);
    if (url.searchParams.get('__theme') !== 'dark') {
        url.searchParams.set('__theme', 'dark');
        window.location.href = url.href;
    }
}
"""
gr.Interface(fn=shout, inputs="textbox", outputs="textbox", flagging_mode="never", js=force_dark_mode).launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


In [9]:
# Inputs and Outputs

view = gr.Interface(
    fn=shout,
    inputs=[gr.Textbox(label="Your message:", lines=6)],
    outputs=[gr.Textbox(label="Response:", lines=8)],
    flagging_mode="never"
)
view.launch()

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


In [10]:
# And now - changing the function from "shout" to "message_gpt"

view = gr.Interface(
    fn=message_gpt,
    inputs=[gr.Textbox(label="Your message:", lines=6)],
    outputs=[gr.Textbox(label="Response:", lines=8)],
    flagging_mode="never"
)
view.launch()

* Running on local URL:  http://127.0.0.1:7863
* To create a public link, set `share=True` in `launch()`.


In [29]:
system_message = "You are a helpful assistant that responds in markdown"

In [30]:
# Let's create a call that streams back results
# If you'd like a refresher on Generators (the "yield" keyword),
# Please take a look at the Intermediate Python notebook in week1 folder.

def stream_gpt(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
      ]
    stream = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=messages,
        stream=True
    )
    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

In [31]:
view = gr.Interface(
    fn=stream_gpt,
    inputs=[gr.Textbox(label="Your message:")],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode="never"
)
view.launch()

* Running on local URL:  http://127.0.0.1:7864
* To create a public link, set `share=True` in `launch()`.


### Projetc on Building a Company Broche Generator 

In [11]:
import requests
from bs4 import BeautifulSoup

class JobPost :
    url: str
    title: str 
    text : str 

    def __init__(self,url):
        self.url = url
        response = requests.get(url)
        soup = BeautifulSoup(response.content,'html.parser')

        for tag in soup(['script','style','img','nav','footer','header']):
            tag.decompose()

        self.text = soup.getText(separator= '\n',strip= True)

        if"responsibilities" in self.text.lower():
            start = self.text.lower().find('responsibilities')
            self.text = self.text[start:]
        
    def get_description(self):
        return self.text

In [12]:
url = "https://careers.gene.com/us/en/job/202506-114830/Data-Engineer"
job = JobPost(url)
print(job.get_description())


Data Engineer in New York City, New York, United States of America | Data Science & AI / ML at Genentech
LOOKING FOR A CONTRACT JOB?
Browse contract jobs via an external website hosted by Magnit
Browse contract jobs
Close the popup
Loading...
Disable Audio Description
Enable Audio Description
Transcript
Who we are
Genentech, a member of the Roche group and founder of the biotechnology industry, is dedicated to pursuing groundbreaking science to discover and develop medicines for people with serious and life-threatening diseases.
To solve the world's most complex health challenges, we ask bigger questions that challenge our industry and the boundaries of science to transform society. Our transformational discoveries include the first targeted antibody for cancer and the first medicine for primary progressive multiple sclerosis.
We celebrate our unique history as the pioneers of scientific breakthroughs that have an incredible track record of improving lives. And yet, our ambitions are b

In [13]:
import fitz
from docx import Document 


def Extract_resum(file):
    if file.name.endswith('.pdf'):
        doc = fitz.open(file.name)
        text = ''
        for page in doc :
            text += page.get_text()
        return text 
    

In [14]:
system_message = "You are an expert career coach. Your job is to rewrite resumes to match job descriptions, optimizing for keywords, role alignment, and tone. Output in plain text."

In [15]:
def stream_gpt(prompt):
    import openai
    system_message = {
        "role": "system",
        "content": "You are an expert career coach. Your job is to rewrite resumes to match job descriptions, optimizing for keywords, role alignment, and tone. Output in plain text."
    }
    user_message = {"role": "user", "content": prompt}

    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[system_message, user_message],
        temperature=0.7,
        stream=True
    )
    for chunk in response:
        if 'choices' in chunk and chunk['choices'][0]['delta'].get('content'):
            yield chunk['choices'][0]['delta']['content']

In [16]:
def Stream_resume(job_URL,resume):
    job = JobPost(job_URL)
    job_text = job.get_description()
    resume_text = Extract_resum(resume)


    prompt = f"""Here is a job post:\n{job_text}\n\nHere is my current resume:\n{resume_text}\n\n
          Please rewrite the resume to better match the job post. Keep it concise and targeted."""
    
    return Stream_resume(prompt)


In [17]:
gr.Interface(
    fn=Stream_resume,
    inputs=[
        gr.Textbox(label="Job Post URL", lines=1, interactive=True,placeholder="Paste job link here"),
        gr.File(label="Upload Resume (PDF)"),
    ],
    outputs=gr.Textbox(label="Tailored Resume"),
    flagging_mode="never"
).launch()

* Running on local URL:  http://127.0.0.1:7864
* To create a public link, set `share=True` in `launch()`.
